In [1]:
#!pip install -q ../input/pydicom-utilities/pydicom_utilities/pylibjpeg-1.4.0-py3-none-any.whl
#!pip install -q ../input/pydicom-utilities/pydicom_utilities/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [2]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
import time

import pydicom
import pathlib
import glob 
from tqdm import tqdm 
import cv2 as cv

#for x in (os.listdir('/kaggle/working/')):
#    shutil.rmtree(x)

In [3]:
## Parameters
sns.set()

EPOCHS = 50
BATCH_SIZE = 16
#IMAGE_SIZE = (512, 512)
IMG_SIZE = (100,100)
SEED = 42

# set seed
np.random.seed(SEED)
tf.random.set_seed(SEED)



In [4]:
def load_dicom(path):
    img=pydicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [5]:
def data_generator():
    """
    a function that will load the dataset from a list of image paths
    """
    for path in img_list:
        data = load_dicom(path)
        yield data  # return the data has generator

In [6]:
DIR='../input/rsna-2022-cervical-spine-fracture-detection/'

train_images_dir=DIR+'train_images/'  
test_images_dir=DIR+'test_images/'  
segmentations_dir=DIR+'segmentations/'  

train_images_list=os.listdir(train_images_dir)
test_images_list=os.listdir(test_images_dir)
segmentations_list=os.listdir(segmentations_dir)

print(len(train_images_list))
train_images_list[0:3]   

2019


['1.2.826.0.1.3680043.17625',
 '1.2.826.0.1.3680043.3850',
 '1.2.826.0.1.3680043.2286']

In [7]:
train_bounding_boxes_csv=pd.read_csv(DIR+'train_bounding_boxes.csv')
sample_submission_csv=pd.read_csv(DIR+'sample_submission.csv')
test_csv=pd.read_csv(DIR+'test.csv')

train_csv=pd.read_csv(DIR+'train.csv')

display(train_csv.info())
display(train_csv)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019 entries, 0 to 2018
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   StudyInstanceUID  2019 non-null   object
 1   patient_overall   2019 non-null   int64 
 2   C1                2019 non-null   int64 
 3   C2                2019 non-null   int64 
 4   C3                2019 non-null   int64 
 5   C4                2019 non-null   int64 
 6   C5                2019 non-null   int64 
 7   C6                2019 non-null   int64 
 8   C7                2019 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 142.1+ KB


None

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0


In [8]:
print('sample_submission.csv  , ',sample_submission_csv.shape)
display(sample_submission_csv)
print('')
print('test.csv  , ', test_csv.shape)
test_csv

sample_submission.csv  ,  (3, 2)


,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.5
1,1.2.826.0.1.3680043.10454_C1,0.5
2,1.2.826.0.1.3680043.10690_C1,0.5



test.csv  ,  (3, 3)


,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.10197_C1,1.2.826.0.1.3680043.10197,C1
1,1.2.826.0.1.3680043.10454_C1,1.2.826.0.1.3680043.10454,C1
2,1.2.826.0.1.3680043.10690_C1,1.2.826.0.1.3680043.10690,C1


## DATA TEST

In [9]:
test_images_dir=DIR + f"test_images"

#os.listdir(test_images_dir)
test_df=pd.DataFrame({'StudyInstanceUID':os.listdir(test_images_dir)})
test_df

Jum_dmc=[]
PatientID=[]
InstanceNumber=[]
BS=[]
TransferSyntaxUID=[]
PhotometricInterpretation=[]
RowsColumns=[]
PixelSpacing=[]
WindowCenter=[]
WindowWidth=[]
PixelRepresentation=[]

for pan in range (len(test_df)):
    w=test_df.loc[pan]['StudyInstanceUID']
    pat = DIR+'test_images/'+w 
    z=os.listdir(pat)[0]
    
    Jum_dmc.append(len(os.listdir(pat)))
    PatientID.append(pydicom.read_file(pat+'/'+z).PatientID)
    #InstanceNumber.append(pydicom.read_file(pat+'/'+z).InstanceNumber)
    BS.append(pydicom.read_file(pat+'/'+z).BitsStored)
    TransferSyntaxUID.append(pydicom.read_file(pat+'/'+z).file_meta.TransferSyntaxUID.name)
    PhotometricInterpretation.append(pydicom.read_file(pat+'/'+z).PhotometricInterpretation)
    RowsColumns.append(str(pydicom.read_file(pat+'/'+z).Rows)+'x'+str(pydicom.read_file(pat+'/'+z).Columns))
    PixelSpacing.append(pydicom.read_file(pat+'/'+z).PixelSpacing)
    WindowCenter.append(pydicom.read_file(pat+'/'+z).WindowCenter)
    WindowWidth.append(pydicom.read_file(pat+'/'+z).WindowWidth)
    PixelRepresentation.append(pydicom.read_file(pat+'/'+z).PixelRepresentation)
    
test_df['PatientID']=PatientID
#test_df['InstanceNumber']=InstanceNumber
test_df['Jum_dmc']=len(os.listdir(pat))
test_df['TransferSyntaxUID']=TransferSyntaxUID
test_df['PhotometricInterpretation']=PhotometricInterpretation
test_df['BitsStored']=BS
test_df['RowsColumns']=RowsColumns
test_df['PixelSpacing']=PixelSpacing
test_df['WindowCenter']=WindowCenter
test_df['WindowWidth']=WindowWidth
test_df['PixelRepresentation']=PixelRepresentation

test_df

,StudyInstanceUID,PatientID,Jum_dmc,TransferSyntaxUID,PhotometricInterpretation,BitsStored,RowsColumns,PixelSpacing,WindowCenter,WindowWidth,PixelRepresentation
0,1.2.826.0.1.3680043.22327,22327,458,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.384766, 0.384766]",550.0,2000.0,1
1,1.2.826.0.1.3680043.25399,25399,458,"JPEG Lossless, Non-Hierarchical, First-Order P...",MONOCHROME2,12,512x512,"[0.361328125, 0.361328125]","[410, 50]","[3665, 350]",0
2,1.2.826.0.1.3680043.5876,5876,458,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.320, 0.320]",500.0,2500.0,1


## DATA TRAIN

In [10]:

train_csv_=pd.read_csv(DIR+'train.csv')
lim1=0   #0    #350  #300    #250    #0    220
lim2=1000000  #250  #400  #350    #300    #220  250


Jum_dcm=[]
PatientID=[]
InstanceNumber=[]
BS=[]
TransferSyntaxUID=[]
PhotometricInterpretation=[]
RowsColumns=[]
PixelSpacing=[]
WindowCenter=[]
WindowWidth=[]
PixelRepresentation=[]

for pan in range (len(train_csv_)):
    w=train_csv_.loc[pan]['StudyInstanceUID']
    
    pat = DIR+'train_images/'+w 
    z=os.listdir(pat)[0]

    Jum_dcm.append(len(os.listdir(pat)))
    
    PatientID.append(pydicom.read_file(pat+'/'+z).PatientID)
    TransferSyntaxUID.append(pydicom.read_file(pat+'/'+z).file_meta.TransferSyntaxUID.name)
    PhotometricInterpretation.append(pydicom.read_file(pat+'/'+z).PhotometricInterpretation)
    BS.append(pydicom.read_file(pat+'/'+z).BitsStored)
    RowsColumns.append(str(pydicom.read_file(pat+'/'+z).Rows)+'x'+str(pydicom.read_file(pat+'/'+z).Columns))
    PixelSpacing.append(pydicom.read_file(pat+'/'+z).PixelSpacing)
    WindowCenter.append(pydicom.read_file(pat+'/'+z).WindowCenter)
    WindowWidth.append(pydicom.read_file(pat+'/'+z).WindowWidth)
    PixelRepresentation.append(pydicom.read_file(pat+'/'+z).PixelRepresentation)
    
train_csv_['Jum_dcm']=Jum_dcm

# INSERT:
train_csv_['PatientID']=PatientID

train_csv_['TransferSyntaxUID']=TransferSyntaxUID
train_csv_['PhotometricInterpretation']=PhotometricInterpretation
train_csv_['BitsStored']=BS
train_csv_['RowsColumns']=RowsColumns
train_csv_['PixelSpacing']=PixelSpacing
train_csv_['WindowCenter']=WindowCenter
train_csv_['WindowWidth']=WindowWidth
train_csv_['PixelRepresentation']=PixelRepresentation

# SYARAT-SYARAT:
train_csv_=train_csv_.loc[train_csv_.Jum_dcm>lim1].loc[train_csv_.Jum_dcm<lim2]
train_csv_=train_csv_.loc[train_csv_.patient_overall==1]
train_csv_=train_csv_.loc[train_csv_.RowsColumns=='512x512']
train_csv_=train_csv_.loc[train_csv_.TransferSyntaxUID!='Implicit VR Little Endian']
train_csv_=train_csv_.loc[train_csv_.PhotometricInterpretation=='MONOCHROME2']
train_csv_=train_csv_.loc[train_csv_.TransferSyntaxUID!='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])']
train_csv_=train_csv_.loc[train_csv_.BitsStored==16]
train_csv_=train_csv_.loc[train_csv_.PixelRepresentation==1]


#print('Kisaran Jum_dcm :',lim1,'-',lim2)
print('Jumlah_dcm/pasien , min:',train_csv_.Jum_dcm.min(), ' , max:',train_csv_.Jum_dcm.max(), ' , Total data dcm:',train_csv_.Jum_dcm.sum())
print('')

train_csv_=train_csv_.sort_values(by=['StudyInstanceUID'],ignore_index=True)
display(train_csv_) #.head())

img_co=train_csv_.Jum_dcm.sum()
print('\t\t\t\t\t\t      ','\033[91m'+'Total data dcm :    ' +str(img_co) + '\033[0m')
print('')
print('img_count:',img_co)
print('')
print('C1=1:',train_csv_.loc[train_csv_.C1==1]['C1'].count(),'dari',train_csv_.C1.count())
print('C1=0:',train_csv_.loc[train_csv_.C1==0]['C1'].count(),'dari',train_csv_.C1.count())


Jumlah_dcm/pasien , min: 69  , max: 825  , Total data dcm: 161442



,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,Jum_dcm,PatientID,TransferSyntaxUID,PhotometricInterpretation,BitsStored,RowsColumns,PixelSpacing,WindowCenter,WindowWidth,PixelRepresentation
0,1.2.826.0.1.3680043.10016,1,0,1,0,0,0,0,0,645,10016,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.275391, 0.275391]",300.0,2000.0,1
1,1.2.826.0.1.3680043.10051,1,0,0,0,1,0,0,0,272,10051,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.253906, 0.253906]",400.0,2500.0,1
2,1.2.826.0.1.3680043.10204,1,1,1,0,0,0,0,0,331,10204,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.416016, 0.416016]",500.0,2000.0,1
3,1.2.826.0.1.3680043.10261,1,1,1,0,0,0,0,0,308,10261,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.544921994209, 0.544921994209]",752.0,2706.0,1
4,1.2.826.0.1.3680043.10400,1,0,0,0,1,1,0,0,270,10400,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.332031, 0.332031]",500.0,2000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,1.2.826.0.1.3680043.9796,1,0,0,0,0,0,1,1,310,9796,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.302734, 0.302734]",250.0,2500.0,1
438,1.2.826.0.1.3680043.9886,1,1,0,0,0,0,0,0,329,9886,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.332031, 0.332031]",500.0,2000.0,1
439,1.2.826.0.1.3680043.9904,1,0,0,0,0,1,0,0,275,9904,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.292969, 0.292969]",550.0,2000.0,1
440,1.2.826.0.1.3680043.9940,1,0,0,0,1,1,0,0,259,9940,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.234375, 0.234375]",550.0,2000.0,1


						       Total data dcm :    161442

img_count: 161442

C1=1: 71 dari 442
C1=0: 371 dari 442


In [11]:
tbb=pd.DataFrame({'StudyInstanceUID':train_bounding_boxes_csv.StudyInstanceUID.unique()})
#train_csv_=pd.read_csv(DATA_DIR+'train.csv').copy()
train_csv_tbb=train_csv_.set_index('StudyInstanceUID').join(tbb.set_index('StudyInstanceUID'), how="inner").reset_index()

#img_count=train_csv_tbb.Jum_dcm.sum()

print('Kisaran Jum_dcm :',lim1,'-',lim2)
print('Jumlah_dcm/pasien , min:',train_csv_tbb.Jum_dcm.min(), ' , max:',train_csv_tbb.Jum_dcm.max(), ' , Total data dcm:',train_csv_tbb.Jum_dcm.sum())
print('img_count:',train_csv_tbb.Jum_dcm.sum())
print('')

display(train_csv_tbb)
img_co=train_csv_tbb.Jum_dcm.sum()
print('\t\t\t\t\t     ','\033[91m'+'Total data dcm :    ' +str(img_co) + '\033[0m')
print('')
print('img_count:',img_co)
print('')
print('C1=1:',train_csv_tbb.loc[train_csv_.C1==1]['C1'].count(),'dari',train_csv_tbb.C1.count())
print('C1=0:',train_csv_tbb.loc[train_csv_.C1==0]['C1'].count(),'dari',train_csv_tbb.C1.count())
print('')

Kisaran Jum_dcm : 0 - 1000000
Jumlah_dcm/pasien , min: 69  , max: 681  , Total data dcm: 28902
img_count: 28902



,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,Jum_dcm,PatientID,TransferSyntaxUID,PhotometricInterpretation,BitsStored,RowsColumns,PixelSpacing,WindowCenter,WindowWidth,PixelRepresentation
0,1.2.826.0.1.3680043.10051,1,0,0,0,1,0,0,0,272,10051,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.253906, 0.253906]",400.0,2500.0,1
1,1.2.826.0.1.3680043.10815,1,0,0,0,1,1,1,0,242,10815,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.265625, 0.265625]",350.0,2000.0,1
2,1.2.826.0.1.3680043.11227,1,1,0,0,0,0,0,0,294,11227,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.341797, 0.341797]",250.0,2500.0,1
3,1.2.826.0.1.3680043.12328,1,0,0,0,0,0,1,0,273,12328,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.281250, 0.281250]",400.0,2500.0,1
4,1.2.826.0.1.3680043.12533,1,0,1,0,0,0,0,1,278,12533,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.292969, 0.292969]",400.0,2500.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,1.2.826.0.1.3680043.8198,1,1,0,0,0,0,0,0,232,8198,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.271484, 0.271484]",400.0,2500.0,1
81,1.2.826.0.1.3680043.8511,1,0,0,0,0,0,1,0,384,8511,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.462891, 0.462891]",500.0,2000.0,1
82,1.2.826.0.1.3680043.8519,1,1,0,0,0,0,0,0,249,8519,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.248047, 0.248047]",550.0,2000.0,1
83,1.2.826.0.1.3680043.8693,1,0,0,0,0,0,1,0,69,8693,Explicit VR Little Endian,MONOCHROME2,16,512x512,"[0.488281, 0.488281]",350.0,2000.0,1


					      Total data dcm :    28902

img_count: 28902

C1=1: 18 dari 85
C1=0: 67 dari 85



In [12]:
df_dcm=pd.DataFrame(train_csv_tbb.sort_values(by=['Jum_dcm'],ignore_index=True)['Jum_dcm']).copy() 
A=df_dcm.Jum_dcm.unique().tolist()   #.size
B=df_dcm.groupby(['Jum_dcm'])['Jum_dcm'].sum().values  #.size
C=pd.DataFrame({'Jum_dcm':A, 'Group_dcm':B})
print(C.Group_dcm.min(), C.Group_dcm.max(),C.Group_dcm.sum())
C

69 1258 28902


,Jum_dcm,Group_dcm
0,69,69
1,173,173
2,229,229
3,232,232
4,238,238
...,...,...
66,649,649
67,661,661
68,665,665
69,673,673


In [13]:
stop

NameError: name 'stop' is not defined

In [ ]:

#train_images_dir=DIR+'train_images/'
#os.listdir(train_images_dir)[0:2]

#covert dcm to jpg/png
import cv2
from PIL import Image
IMG_SIZE=200

# Specify the .dcm folder path

train_JPEG_BS_=train_csv_tbb.copy()

folder_path = train_JPEG_BS_.StudyInstanceUID.values.tolist()  #[0:2]
f_dest='train_images_'+str(lim1)+'_'+str(lim2)

try: 
    os.mkdir(f_dest)
except OSError as error: 
    print(error)

i=0 
for n, image in enumerate(folder_path[0:]):  #2018
    #print('image:',image)
    
    fold=train_JPEG_BS_.loc[train_JPEG_BS_.StudyInstanceUID==str(image)].C1.values[0]
    
    try: 
        os.mkdir(f_dest+'/'+str(fold))
    except OSError as error: 
        print(error)  
        
    for img in os.listdir(train_images_dir+image)[0:]:       
        ds = pydicom.read_file(train_images_dir+image+'/'+img)
        #ds=load_dicom(os.path.join(train_images_dir+image+'/'+img))

        pixel_array_numpy = ds.pixel_array
        pixel_array_numpy = cv.resize(pixel_array_numpy, (IMG_SIZE, IMG_SIZE))
  
        img_mem = Image.fromarray(pixel_array_numpy)
        #print('img_mem:',img_mem)
        #img_mem.save(f_dest+'/'+str(image)+'/'+img.replace(".dcm",".png"))
               
        img_mem.save(f_dest+'/'+str(fold)+'/'+str(image)+'.'+str(img).replace(".dcm",".png"))
        #print('f_dest:',f_dest+'/'+str(fold)+'/'+str(image)+'.'+str(img).replace(".dcm",".png"))
        
        if i % 500 == 0:
            #print('   ',train_images_dir+image+'/'+img)
            print('{} image converted to png'.format(i))
            print('   f_dest:',f_dest+'/'+str(fold)+'/'+str(image)+'.'+str(img).replace(".dcm",".png"))
        i+=1
        #print()

<a id="makedf"></a>
# <center>Read in images and create a dataframe of image paths and class labels</center>

In [ ]:
#shutil.rmtree('aug')

DIR='../input/rsna-2022-cervical-spine-fracture-detection/'
sdir=(DIR+'train_images/')

min_samples=0 # set limit for minimum images a class must have to be included in the dataframe
filepaths = []
labels=[] 
classlist=os.listdir(sdir)[0:]   
#for klass in classlist:
#for klass in train_JPEG_BS_.StudyInstanceUID.tolist()[0:]:
for klass in (os.listdir('/kaggle/working/train_images_'+str(lim1)+'_'+str(lim2))):
    
    classpath=os.path.join('/kaggle/working/train_images_'+str(lim1)+'_'+str(lim2), klass)
    flist=os.listdir(classpath)
    
    #print('klass:',klass)
    
    #path_0=str(classpath)+'/'+flist[1]
    #print('path_0:',path_0)
    #dicom_file = pydicom.read_file(path_0)
    #tx_meta='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])'
    #if dicom_file.file_meta.TransferSyntaxUID.name == tx_meta or dicom_file.BitsStored!=16:
    #    print('Not included:',len(flist),'file dcm ,folder: ',classpath)
    #    continue
            
    if len(flist) >= min_samples:
        for f in flist:
            #print('f:',f)
            fpath=os.path.join(classpath,f)
            #print('fpath:',fpath)
            #dicom_file = pydicom.read_file(fpath)
            #if dicom_file.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' or dicom_file.BitsStored!=16:
            #    continue
            #if dicom_file.BitsStored!=16:
            #    continue
            #img = load_dicom(fpath)        

            filepaths.append(fpath)
            labels.append(klass)
            #lab=train_JPEG_BS_.loc[train_JPEG_BS_.StudyInstanceUID == klass,['patient_overall','C1','C2','C3','C4','C5','C6','C7']].values[0].tolist()
            #print('lab:',lab)
            #labels.append(train_JPEG_BS_.loc[train_JPEG_BS_.StudyInstanceUID == klass,['patient_overall','C1','C2','C3','C4','C5','C6','C7']].values[0].tolist())
            
            #print('fpath:',fpath)
            #print('labels:',labels)
    else:
        print('class ', klass, ' has only', len(flist), ' samples and will not be included in dataframe')
        
Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')        
df=pd.concat([Fseries, Lseries], axis=1)
#display("df:",df)

train_df, dummy_df=train_test_split(df, train_size=.9, shuffle=True, random_state=123, stratify=df['labels'])
valid_df, test_df=train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123, stratify=dummy_df['labels'])

print('train_df lenght: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
# get the number of classes and the images count for each class in train_df

classes=sorted(list(train_df['labels'].unique()))
class_count = len(classes)
print('The number of classes in the dataset is: ', class_count)
groups=train_df.groupby('labels')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
countlist=[]
classlist=[]
no=0
for label in sorted(list(train_df['labels'].unique())):
    group=groups.get_group(label)
    countlist.append(len(group))
    classlist.append(label)
    print(no,'{0:^30s} {1:^13s}'.format(label, str(len(group))))
    no+=1

# get the classes with the minimum and maximum number of train images
max_value=np.max(countlist)
max_index=countlist.index(max_value)
max_class=classlist[max_index]
min_value=np.min(countlist)
min_index=countlist.index(min_value)
min_class=classlist[min_index]
print(max_class, ' has the most images= ',max_value, ' ', min_class, ' has the least images= ', min_value)
# lets get the average height and width of a sample of the train images
ht=0
wt=0
# select 100 random samples of train_df
train_df_sample=train_df.sample(n=200, random_state=123,axis=0)
for i in range (len(train_df_sample)):
    fpath=train_df_sample['filepaths'].iloc[i]
    #print('fpath:',fpath)
    img=cv2.imread(fpath)
    shape=img.shape
    ht += shape[0]
    wt += shape[1]
print('average height= ', ht//100, ' average width= ', wt//100, 'aspect ratio= ', ht/wt)

<a id="trim"></a>
# <center>Trim train_df so no class has more than 200 images</center>

In [ ]:
def trim(df, max_samples, min_samples, column):
    df=df.copy()
    groups=df.groupby(column)    
    trimmed_df = pd.DataFrame(columns = df.columns)
    groups=df.groupby(column)
    for label in df[column].unique(): 
        group=groups.get_group(label)
        count=len(group)    
        if count > max_samples:
            sampled_group=group.sample(n=max_samples, random_state=123,axis=0)
            trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
        else:
            if count>=min_samples:
                sampled_group=group        
                trimmed_df=pd.concat([trimmed_df, sampled_group], axis=0)
    print('after trimming, the maximum samples in any class is now ',max_samples, ' and the minimum samples in any class is ', min_samples)
    return trimmed_df

max_samples=100 # since each class has more than 200 images all classes will be trimmed to have 200 images per class
min_samples=36
column='labels'
train_df= trim(train_df, max_samples, min_samples, column)

<a id="balance"></a>
# <center>Balance train_df by creating augmented images</center>


In [ ]:
def balance(df, n, working_dir, img_size):
    df=df.copy()
    print('Initial length of dataframe is ', len(df))
    
    aug_dir=os.path.join(working_dir, 'aug')# directory to store augmented images
    print('aug_dir:',aug_dir)
    if os.path.isdir(aug_dir):# start with an empty directory
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)        
    for label in df['labels'].unique():    
        dir_path=os.path.join(aug_dir,label)    
        os.mkdir(dir_path) # make class directories within aug directory
    # create and store the augmented images  
    total=0
    gen=ImageDataGenerator(horizontal_flip=True,  rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
    groups=df.groupby('labels') # group by class
    print('groups:',groups)
    for label in df['labels'].unique():  # for every class               
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
        sample_count=len(group)   # determine how many samples there are in this class  
        #print('sample_count:',sample_count)
        #print('n:',n)
        if sample_count< n: # if the class has less than target number of images
            aug_img_count=0
            delta=n - sample_count  # number of augmented images to create
            print('delta:',delta)
            target_dir=os.path.join(aug_dir, label)  # define where to write the images
            print('target_dir:',target_dir)
            msg='{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='') # prints over on the same line
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=img_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                            save_format='png') #'dcm') #'jpg')
            while aug_img_count<delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total +=aug_img_count
    print('Total Augmented images created= ', total)
    # create aug_df and merge with train_df to create composite training set ndf
    aug_fpaths=[]
    aug_labels=[]
    classlist=os.listdir(aug_dir)
    for klass in classlist:
        classpath=os.path.join(aug_dir, klass)     
        flist=os.listdir(classpath)    
        for f in flist:        
            fpath=os.path.join(classpath,f)         
            aug_fpaths.append(fpath)
            aug_labels.append(klass)
    Fseries=pd.Series(aug_fpaths, name='filepaths')
    Lseries=pd.Series(aug_labels, name='labels')
    aug_df=pd.concat([Fseries, Lseries], axis=1)         
    df=pd.concat([df,aug_df], axis=0).reset_index(drop=True)
    print('Length of augmented dataframe is now ', len(df))
    return df 

n=200 # number of samples in each class
working_dir=r'./' # directory to store augmented images
img_size=(IMG_SIZE,IMG_SIZE)  #(200,200) # size of augmented images
train_df=balance(train_df, n, working_dir, img_size)    

<a id="generators"></a>
# <center>Create the train_gen, test_gen final_test_gen and valid_gen</center>

In [ ]:
batch_size=20 # We will use and EfficientetB3 model, with image size of (200, 250) this size should not cause resource error
trgen=ImageDataGenerator(horizontal_flip=True,rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
t_and_v_gen=ImageDataGenerator()
msg='{0:70s} for train generator'.format(' ')
print(msg, '\r', end='') # prints over on the same line

train_gen=trgen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
msg='{0:70s} for valid generator'.format(' ')
print(msg, '\r', end='') # prints over on the same line
valid_gen=t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)
# for the test_gen we want to calculate the batch size and test steps such that batch_size X test_steps= number of samples in test set
# this insures that we go through all the sample in the test set exactly once.
length=len(test_df)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]  
test_steps=int(length/test_batch_size)
msg='{0:70s} for test generator'.format(' ')
print(msg, '\r', end='') # prints over on the same line

test_gen=t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)
# from the generator we can get information we will need later
classes=list(train_gen.class_indices.keys())
class_indices=list(train_gen.class_indices.values())
class_count=len(classes)
labels=test_gen.labels
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps, ' number of classes : ', class_count)

<a id="show"></a>
# <center>Create a function to show example training images</center>

In [ ]:
def show_image_samples(gen ):
    t_dict=gen.class_indices
    classes=list(t_dict.keys())    
    images,labels=next(gen) # get a sample batch from the generator 
    plt.figure(figsize=(20, 20))
    length=len(labels)
    if length<25:   #show maximum of 25 images
        r=length
    else:
        r=25
    for i in range(r):        
        plt.subplot(5, 5, i + 1)
        
        image=images[i] /255       
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color='blue', fontsize=14)
        plt.axis('off')
    plt.show()
    
show_image_samples(train_gen )

<a id="model"></a>
# <center>Create a model using transfer learning with EfficientNetB3</center>
### NOTE experts advise you make the base model initially not trainable. Then train for some number of epochs
### then fine tune model by making base model trainable and run more epochs
### I have found this to be WRONG!!!!
### Making the base model trainable from the outset leads to faster convegence and a lower validation loss
### for the same number of total epochs!

In [ ]:
img_shape=(img_size[0], img_size[1], 3)
model_name='EfficientNetB0'
base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 

# Note you are always told NOT to make the base model trainable initially- that is WRONG you get better results leaving it trainable
base_model.trainable=True
x=base_model.output
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.4, seed=123)(x)       
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=.001 # start with this learning rate
model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 


<a id="callback"></a>
# <center>Create a custom Keras callback to continue and optionally set LR or halt training</center>
The LR_ASK callback is a convenient callback that allows you to continue training for ask_epoch more epochs or to halt training.  
If you elect to continue training for more epochs you are given the option to retain the current learning rate (LR) or to  
enter a new value for the learning rate. The form of use is:  
ask=LR_ASK(model,epochs, ask_epoch) where:  
* model is a string which is the name of your compiled model
* epochs is an integer which is the number of epochs to run specified in model.fit
* ask_epoch is an integer. If ask_epoch is set to a value say 5 then the model will train for 5 epochs.  
  then the user is ask to enter H to halt training, or enter an inter value. For example if you enter 4  
  training will continue for 4 more epochs to epoch 9 then you will be queried again. Once you enter an  
  integer value you are prompted to press ENTER to continue training using the current learning rate  
  or to enter a new value for the learning rate.  
  
 At the end of training the model weights are set to the weights for the epoch that achieved the lowest validation loss

In [ ]:
class LR_ASK(keras.callbacks.Callback):
    def __init__ (self, model, epochs,  ask_epoch): # initialization of the callback
        super(LR_ASK, self).__init__()
        self.model=model               
        self.ask_epoch=ask_epoch
        self.epochs=epochs
        self.ask=True # if True query the user on a specified epoch
        self.lowest_vloss=np.inf
        self.best_weights=self.model.get_weights() # set best weights to model's initial weights
        self.best_epoch=1
        
        
    def on_train_begin(self, logs=None): # this runs on the beginning of training
        if self.ask_epoch == 0: 
            print('you set ask_epoch = 0, ask_epoch will be set to 1', flush=True)
            self.ask_epoch=1
        if self.ask_epoch >= self.epochs: # you are running for epochs but ask_epoch>epochs
            print('ask_epoch >= epochs, will train for ', epochs, ' epochs', flush=True)
            self.ask=False # do not query the user
        if self.epochs == 1:
            self.ask=False # running only for 1 epoch so do not query user
        else:
            print('Training will proceed until epoch', ask_epoch,' then you will be asked to') 
            print(' enter H to halt training or enter an integer for how many more epochs to run then be asked again')  
        self.start_time= time.time() # set the time at which training started
        
    def on_train_end(self, logs=None):   # runs at the end of training  
        print('loading model with weights from epoch ', self.best_epoch)
        self.model.set_weights(self.best_weights) # set the weights of the model to the best weights
        tr_duration=time.time() - self.start_time   # determine how long the training cycle lasted         
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print (msg, flush=True) # print out training duration time
        
    def on_epoch_end(self, epoch, logs=None):  # method runs on the end of each epoch
        v_loss=logs.get('val_loss')  # get the validation loss for this epoch
        if v_loss< self.lowest_vloss:
            self.lowest_vloss=v_loss
            self.best_weights=self.model.get_weights() # set best weights to model's initial weights
            self.best_epoch=epoch + 1
            print (f'\n validation loss of {v_loss:7.4f} is below lowest loss, saving weights from epoch {str(epoch + 1):3s} as best weights')
        else:
            print (f'\n validation loss of {v_loss:7.4f} is above lowest loss of {self.lowest_vloss:7.4f} keeping weights from epoch {str(self.best_epoch)} as best weights')
        
        if self.ask: # are the conditions right to query the user?
            if epoch + 1 ==self.ask_epoch: # is this epoch the one for quering the user?
                print('\n Enter H to end training or  an integer for the number of additional epochs to run then ask again')
                ans=input()
                
                if ans == 'H' or ans =='h' or ans == '0': # quit training for these conditions
                    print ('you entered ', ans, ' Training halted on epoch ', epoch+1, ' due to user input\n', flush=True)
                    self.model.stop_training = True # halt training
                else: # user wants to continue training
                    self.ask_epoch += int(ans)
                    if self.ask_epoch > self.epochs:
                        print('\nYou specified maximum epochs of as ', self.epochs, ' cannot train for ', self.ask_epoch, flush =True)
                    else:
                        print ('you entered ', ans, ' Training will continue to epoch ', self.ask_epoch, flush=True)
                        lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
                        print(f'current LR is  {lr:7.5f}  hit enter to keep  this LR or enter a new LR')
                        ans=input(' ')
                        if ans =='':
                            print (f'keeping current LR of {lr:7.5f}')
                        else:
                            new_lr=float(ans)
                            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
                            print(' changing LR to ', ans)

<a id="callbacks"></a>
# <center>Instantiate custom callback 

In [ ]:
epochs=50  #200
ask_epoch=60  #100
ask=LR_ASK(model, epochs,  ask_epoch)
#rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,verbose=1)
#callbacks=[rlronp, ask]
callbacks=[ask]

<a id="train"></a>
# <center>Train the model
### Note unlike how you are told it is BETTER to make the base model trainable from the outset
### It will converge faster and have a lower validation losss

In [ ]:
history=model.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

<a id="plot"></a>
# <center>Define a function to plot the training data

In [ ]:
def tr_plot(tr_data, start_epoch):
    #Plot the training and validation data
    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(24,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout    
    plt.show()
    
tr_plot(history,0)

<a id="result"></a>
# <center>Make Predictions on the test set</a>
### Define a function which takes in a test generator and an integer test_steps
### and generates predictions on the test set including a confusion matric
### and a classification report

In [ ]:
def predictor(test_gen, test_steps):
    y_pred= []
    y_true=test_gen.labels
    classes=list(test_gen.class_indices.keys())
    class_count=len(classes)
    errors=0
    preds=model.predict(test_gen, verbose=1)
    tests=len(preds)    
    for i, p in enumerate(preds):        
        pred_index=np.argmax(p)         
        true_index=test_gen.labels[i]  # labels are integer values        
        if pred_index != true_index: # a misclassification has occurred                                           
            errors=errors + 1
        y_pred.append(pred_index)
            
    acc=( 1-errors/tests) * 100 
    print(f'there were {errors} errors in {tests} tests for an accuracy of {acc:6.2f}')
    ypred=np.array(y_pred)
    ytrue=np.array(y_true)
    if class_count <=30:
        cm = confusion_matrix(ytrue, ypred )
        # plot the confusion matrix
        plt.figure(figsize=(4, 3))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
        plt.xticks(np.arange(class_count)+.5, classes, rotation=0)
        plt.yticks(np.arange(class_count)+.5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
    clr = classification_report(y_true, y_pred, target_names=classes, digits= 4) # create classification report
    print("Classification Report:\n----------------------\n", clr)
    return errors, tests
errors, tests=predictor(test_gen, test_steps)

In [ ]:
STOP

<a id="save"></a>
# <center>Save the model

In [ ]:
subject='plant disease' 
acc=str(( 1-errors/tests) * 100)
index=acc.rfind('.')
acc=acc[:index + 3]
save_id= subject + '_' + str(acc) + '.h5' 
model_save_loc=os.path.join('working_dir', save_id)
model.save(model_save_loc)
print ('model was saved as ' , model_save_loc ) 
   

In [ ]:

#LOAD MODEL
#from tensorflow.keras import layers, Sequential
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
"""
from tensorflow.keras.models import load_model

MODEL_PATH = './M-EfficientNetB3_e100.h5'

model = load_model(MODEL_PATH,compile=False) #, custom_objects={'KerasLayer': hub.KerasLayer})
model
"""

## Convert data test to png

In [ ]:
test_csv_=test_csv[0:].copy()
txt_JPEG='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])'
Impli='Implicit VR Little Endian'
Expli='Explicit VR Little Endian'

for pan in range (len(test_csv_)):
    w=test_csv_.loc[pan]['StudyInstanceUID']
    pat = DIR+'train_images/'+w 
    z=os.listdir(pat)[0]
    dicom_file = pydicom.read_file(pat+'/'+z)
    #print(pan,dicom_file.file_meta.TransferSyntaxUID.name , dicom_file.BitsStored)
    
    if dicom_file.file_meta.TransferSyntaxUID.name == Impli:
        test_csv_.loc[pan,'Impli']=1
    else:
        test_csv_.loc[pan,'Impli']=0
        
    if dicom_file.file_meta.TransferSyntaxUID.name == Expli:
        test_csv_.loc[pan,'Expli']=1
    else:
        estn_csv_.loc[pan,'Expli']=0
    
    if dicom_file.file_meta.TransferSyntaxUID.name == txt_JPEG:
        test_csv_.loc[pan,'txt_JPEG']=1
    else:
        test_csv_.loc[pan,'txt_JPEG']=0
        
    if dicom_file.BitsStored == 16:
        test_csv_.loc[pan,'BS_16']=1
    else:
        test_csv_.loc[pan,'BS_16']=0
        
    test_csv_.loc[pan,'Jum_dcm']=len(os.listdir(pat))
test_csv_

test_JPEG_BS =train_csv_.loc[test_csv_.txt_JPEG==0].loc[test_csv_.BS_16==1].loc[test_csv_.patient_overall==1]
display(test_JPEG_BS)
print('\t\t\t\t\t\t\t\t\t\t\t\t\t     ','\033[91m'+'Total data dcm :  ' +str(test_JPEG_BS.Jum_dcm.sum()) + '\033[0m')

#=======================

test_JPEG_BS_=test_JPEG_BS.copy()
#lim1=220
#lim2=250
#test_JPEG_BS_=test_JPEG_BS_.loc[test_JPEG_BS_.Jum_dcm>lim1].loc[test_JPEG_BS_.Jum_dcm<lim2]
#print('min:',test_JPEG_BS_.Jum_dcm.min(), 'max:',test_JPEG_BS_.Jum_dcm.max(), 'sum:',test_JPEG_BS_.Jum_dcm.sum())

test_JPEG_BS_=test_JPEG_BS_.sort_values(by=['StudyInstanceUID'],ignore_index=True)

display(test_JPEG_BS_.head())
img_count=test_JPEG_BS_.Jum_dcm.sum()
print('\t\t\t\t\t\t\t\t\t\t\t\t\t     ','\033[91m'+'Total data dcm :  ' +str(img_count) + '\033[0m')
print('')
print('img_count:',img_count)
print('Cgab.nunique:',test_JPEG_BS_.Cgab.nunique())
#print('Cgab.nunique:',train_JPEG_BS_.C1.nunique())
print('')
print('C1=1:',test_JPEG_BS_.loc[test_JPEG_BS_.C1==1]['C1'].count(),'dari',test_JPEG_BS_.C1.count())
print('C1=0:',test_JPEG_BS_.loc[test_JPEG_BS_.C1==0]['C1'].count(),'dari',test_JPEG_BS_.C1.count())

#=========================

#covert dcm to jpg/png
import cv2
from PIL import Image
IMG_SIZE=200

# Specify the .dcm folder path
folder_path = test_JPEG_BS_.StudyInstanceUID.values.tolist()  #[0:2]
f_dest='test_images'  #+str(lim1)+'_'+str(lim2)

try: 
    os.mkdir(f_dest)
except OSError as error: 
    print(error)

i=0 
for n, image in enumerate(folder_path[0:]):  #2018   
    fold=train_JPEG_BS_.loc[test_JPEG_BS_.StudyInstanceUID==str(image)].C1.values[0]
    
    try: 
        os.mkdir(f_dest+'/'+str(fold))
    except OSError as error: 
        print(error)  
        
    for img in os.listdir(test_images_dir+image)[0:]:       
        ds = pydicom.read_file(test_images_dir+image+'/'+img)        
        pixel_array_numpy = ds.pixel_array
        pixel_array_numpy = cv.resize(pixel_array_numpy, (IMG_SIZE, IMG_SIZE))
        img_mem = Image.fromarray(pixel_array_numpy)               
        img_mem.save(f_dest+'/'+str(fold)+'/'+str(image)+'.'+str(img).replace(".dcm",".png"))
        
        if i % 500 == 0:
            print('{} image converted to png'.format(i))
            print('   f_dest:',f_dest+'/'+str(fold)+'/'+str(image)+'.'+str(img).replace(".dcm",".png"))
        i+=1
        #print()

In [ ]:
classlist[0:5]

In [ ]:
#data_train_lab=['blast','brown','healthy']
data_train_lab=classlist

data_train_nom=[]
for k in range (0,int(len(data_train_lab))):
    data_train_nom.append(k)
data_train_nom

print(data_train_lab)
print(data_train_nom)

In [ ]:
# MEMILIH DATA UNTUK TEST
#data_test=data_dir

def load_image(image_file):
    img = Image.open(image_file)
    return img 

def prediksi(im):
    #input_size = (224,224)
    input_size = (200,200)
    channel = (3,)
    input_shape = input_size + channel
    labels1 = data_train_lab
    labels2 = data_train_nom

    def preprocess(img,input_size):
        nimg = img.convert('RGB').resize(input_size, resample= 0)
        img_arr = (np.array(nimg))/255
        return img_arr

    def reshape(imgs_arr):
        return np.stack(imgs_arr, axis=0)

    #from tensorflow.keras.models import load_model
    #MODEL_PATH = 'model-Zin_1.h5'
    #MODEL_PATH = save_model_path
    #model = load_model(MODEL_PATH,compile=False, custom_objects={'KerasLayer': hub.KerasLayer})
    
    # read image
    #im = Image.open(data_uji)
    X = preprocess(im,input_size)
    X = reshape([X])
    y = model.predict(X)
    
    HasilPrediksi1=labels1[np.argmax(y)],  
    HasilPrediksi2=labels2[np.argmax(y)],
    #print()
    #print('Keterangan = ',labels)
    #print()
    #st.write( 'HasilPrediksi = ', HasilPrediksi)
    return HasilPrediksi1,HasilPrediksi2,y



In [ ]:
sdir=r'../input/plant-disease-expert/Image Data base/'
len(os.listdir(sdir+'Image Data base/'))
os.listdir(sdir+'Image Data base/')
len(os.listdir(sdir+'Image Data base/'))
os.listdir(sdir+'Image Data base/')

In [ ]:
%%time
from PIL import Image
#from google.colab.patches import cv2_imshow

#dImages = '/content/drive/MyDrive/Colab Notebooks/Microsoft Rice Disease Classification Challenge/Data/Images/'
#dImages = 'Data-score-99/TrainImages_no_rgn/'  #maindir2 #'input/ricediseases2/'
dImages = sdir  #+'/Image Data base/'

gamb=os.listdir(dImages)[0] #[0]
image_file = dImages+'/'+gamb
print(gamb)
if image_file is not None:
    print(image_file+'/'+os.listdir(image_file)[0])
    img_uji = load_image(image_file+'/'+os.listdir(image_file)[0])  
    HasilPrediksi1,HasilPrediksi2,y = prediksi(img_uji)
    print('y=',y)
    #print('y0=',y[0][0])
    #print('y1=',y[0][1])
    
    print("HASIL PREDIKSI = ",HasilPrediksi2[0],' (',HasilPrediksi1[0],')')

    img = cv2.imread(image_file+'/'+os.listdir(image_file)[0])
    height, width, channels = img.shape
    print(height, width, channels)
    img=cv2.resize(img, (200,200), interpolation = cv2.INTER_AREA)
    #cv2_imshow(img)
    plt.imshow(img)


In [ ]:
%%time
g=0
#maindir0='Data-score-99/TestImages_no_rgn/'
maindir0=image_file
SampleSubmission=pd.read_csv('Data-score-99/SampleSubmission.csv')

SamSub=SampleSubmission.copy()
SamSub=SamSub.set_index('Image_id')

#for jpg in SampleSubmission['Image_id'][0:]: 
for jpg in image_file+'/'+os.listdir(image_file)#[0]
  #print(dImages+jpg)
  image_file = maindir0

  print(g,' ',jpg)
  if image_file is not None:
      print(maindir0+jpg)  
      img_uji = load_image(maindir0+jpg)  
      HasilPrediksi1,HasilPrediksi2,y = prediksi(img_uji)
      print(g,jpg,y,HasilPrediksi1,HasilPrediksi2)

      SamSub.loc[SamSub.index==jpg,['blast','brown','healthy']]=y    
      g+=1


display(SamSub)

from pathlib import Path  
filepath = Path('Data-score-99/'+str(model_name)+'_e'+str(epochs)+'/Sub_'+str(model_name)+'_e'+str(epochs)+'_no_rgn.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
SamSub.to_csv(filepath)

In [ ]:
SampleSubmission

In [ ]:
from pathlib import Path  
filepath = Path('Data-score-99/'+str(model_name)+'_e'+str(epochs)+'/Sub_'+str(model_name)+'_e'+str(epochs)+'_no_rgn.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
SamSub.to_csv(filepath)